In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import cv2
import os
import PIL.Image
import imquality.brisque as brisque
import math
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import pickle
from sklearn.preprocessing import RobustScaler

In [90]:
gambar = im = PIL.Image.open('train/Rendah/2H3e8RgY7C0.jpg')
def brightness(img):
    width, height = img.size
    r_total = 0
    g_total = 0
    b_total = 0
    for x in range(width):
        for y in range(height):
            r, g, b = img.getpixel((x, y))
            r_total += r**2
            g_total += g**2
            b_total += b**2
    brightness = math.sqrt(0.241 * r_total + 0.691 * g_total + 0.068 * b_total)
    return brightness

In [84]:
type(gambar)

PIL.JpegImagePlugin.JpegImageFile

In [91]:
brightness(gambar)

76194.68705820636

In [21]:
def complexity(img):
  open_cv_image = np.array(img) 
  gambar_abu_abu = cv2.cvtColor(open_cv_image, cv2.COLOR_BGR2GRAY)

# Gunakan algoritma deteksi tepi Canny untuk menemukan tepi dalam gambar
  tepi = cv2.Canny(gambar_abu_abu, 100, 200)
  tepi = tepi.reshape(-1,1)

# Gunakan Random Forest classifier untuk mengklasifikasi tepi sebagai sederhana atau kompleks
  classifier = RandomForestClassifier()
  classifier.fit(tepi, [0 if edge > 0 else 1 for edge in tepi])

# Hitung skor kompleksitas komponen dengan mencari rata-rata prediksi classifier
  skor_kompleksitas = classifier.predict(tepi).mean()
  return skor_kompleksitas

In [22]:
complexity(gambar)

0.9810405815972222

In [24]:
def colorfulness(img):
  open_cv_image = np.array(img) 
  (B, G, R) = cv2.split(open_cv_image.astype("float"))
	# compute rg = R - G
  rg = np.absolute(R - G)
    # compute yb = 0.5 * (R + G) - B
  yb = np.absolute(0.5 * (R + G) - B)
  # compute the mean and standard deviation of both `rg` and `yb`
  (rbMean, rbStd) = (np.mean(rg), np.std(rg))
  (ybMean, ybStd) = (np.mean(yb), np.std(yb))
  # combine the mean and standard deviations
  stdRoot = np.sqrt((rbStd ** 2) + (ybStd ** 2))
  meanRoot = np.sqrt((rbMean ** 2) + (ybMean ** 2))
  # derive the "colorfulness" metric and return it
  score = stdRoot + (0.3 * meanRoot)
  return score

In [25]:
colorfulness(gambar)

111.6722177725027

In [27]:
def brisque_score(images):
    score = brisque.score(images)
    return score

In [28]:
brisque_score(gambar)

c:\Users\farras\anaconda3\lib\site-packages\imquality\brisque.py:45: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
  self.image = skimage.color.rgb2gray(self.image)


77.19267077045399

In [42]:
from tensorflow.keras.models import Sequential, model_from_json
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [67]:
df = pd.read_csv("data_regresi.csv")

# Separate Target Variable and Predictor Variables
TargetVariable=['Detail Views']
Predictors=['brightness','complexity','colorfulness','quality']

X=df[Predictors].values
y=df[TargetVariable].values
### Sandardization of data ###
from sklearn.preprocessing import StandardScaler
PredictorScaler=StandardScaler()
TargetVarScaler=StandardScaler()

# Storing the fit object for later reference
PredictorScalerFit=PredictorScaler.fit(X)
TargetVarScalerFit=TargetVarScaler.fit(y)

# Generating the standardized values of X and y
X=PredictorScalerFit.transform(X)
y=TargetVarScalerFit.transform(y)

In [ ]:
from sklearn.preprocessing import RobustScaler

# fit and transform the data
df_x = df[['brightness', 'complexity', 'colorfulness','quality']]
df_robust = pd.DataFrame(scaler.fit_transform(df_x), columns=df_x.columns)
df_robust

In [92]:
def classify_image(inp):
    IMAGE_SIZE = (640, 360)
    attr = []
    
    attr.append(brightness(inp))
    
    inp = np.array(inp)
    inp = cv2.cvtColor(inp, cv2.COLOR_BGR2RGB)
    inp = cv2.resize(inp, IMAGE_SIZE)
    
    attr.append(complexity(inp))
    attr.append(colorfulness(inp))
    attr.append(brisque_score(inp))
    attr = np.array(attr).reshape(-1, 1)
    
    robust = RobustScaler()
    attr_robust = robust.fit_transform(attr)
    
    scaled = PredictorScaler.fit_transform(attr_robust)
    scaled = scaled.reshape(scaled.shape[1], scaled.shape[0])
    # print(type(scaled))
    # print(scaled.shape)
    # model =  pickle.load(open(r'C:\SAINS DATA\SEMESTER 5\Data Mining 2\UAS\model_regresi_thumbnail.sav', 'rb'))
    
    prediction = loaded_model.predict(scaled)
    prediction_inverse = TargetVarScalerFit.inverse_transform(prediction)
    
    return prediction_inverse[0]

In [ ]:
# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = loaded_model.evaluate(X, Y, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

In [93]:
classify_image(gambar)

c:\Users\farras\anaconda3\lib\site-packages\imquality\brisque.py:45: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
  self.image = skimage.color.rgb2gray(self.image)


1/1 [==============================] - 0s 21ms/step


array([749913.], dtype=float32)

c:\Users\farras\anaconda3\lib\site-packages\imquality\brisque.py:45: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
  self.image = skimage.color.rgb2gray(self.image)


1/1 [==============================] - 0s 18ms/step


In [18]:
import gradio as gr

gr.Interface(fn=classify_image, 
                inputs=gr.Image(shape=(1280, 720)),
                outputs=gr.outputs.Textbox(label='Predicted Score'),
                examples=["banana.jpg", "car.jpg"]).launch()

c:\Users\farras\anaconda3\lib\site-packages\gradio\outputs.py:21: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(


KeyboardInterrupt: 

In [53]:
import gradio as gr

gr.Interface(fn=classify_image, 
                inputs=gr.Image(shape=(1280, 720), type = 'pil'),
                outputs=gr.outputs.Textbox(label='Predicted Score')).launch()

c:\Users\farras\anaconda3\lib\site-packages\gradio\outputs.py:21: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(


Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


c:\Users\farras\anaconda3\lib\site-packages\imquality\brisque.py:45: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
  self.image = skimage.color.rgb2gray(self.image)


1/1 [==============================] - 0s 18ms/step


c:\Users\farras\anaconda3\lib\site-packages\imquality\brisque.py:45: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
  self.image = skimage.color.rgb2gray(self.image)


1/1 [==============================] - 0s 19ms/step
